In [3]:
import os
import csv
from time import *

from greedy import greedy_mwds

from lp_model import lp_model
from genetic_algorithm import GeneticAlgorithm
from numpy import mean, std

In [4]:
import networkx as nx, random
""""
Funkcija koja se koristi za citanje/generisanje grafa
"""

def gen_graph_from_edges(instance_path):
    edges = []
    with open(instance_path, 'r') as f:
        for line in f:
            fields = line.strip().split()
            source, target = fields
            edges.append((int(source), int(target)))

    for i in range(len(edges)):
            edges[i] = (edges[i][0] - 1, edges[i][1] - 1)

    G = nx.Graph(edges)
    weights = [random.randint(20, 70) for i in range(len(G.nodes))]
    vertices_w = dict(zip(G.nodes, weights))

    return vertices_w, edges

In [5]:
dir_instances = sorted(os.listdir("../instances"))

In [2]:
for subdir in dir_instances:
    dim_path = os.path.join("../instances/", subdir)

    if subdir.endswith("0"):
        instance_dim = "0"
    elif subdir.endswith("1"):
        instance_dim = "1"
    elif subdir.endswith("2"):
        instance_dim = "2"


    with open("../results/results" + instance_dim + ".csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)

        # header
        writer.writerow(["Instances", "Greedy algorithm", "Genetic algorithm", "PuLP linear programming"])
        instance_info = ""

        # prolazak kroz sve instance odredjene dimenzije
        for instance in sorted(os.listdir(os.path.abspath(dim_path))):
            if instance != "imgs":
                instance_path = os.path.join(dim_path, instance)

                if dim_path.endswith("0"):
                    num_vertices = 5
                    instance_info = instance.split(".txt")[0]
                else:
                    if instance.__contains__("200"):
                        num_vertices = 200
                        instance_info = instance.split(".clq")[0]
                        break
                    elif instance.__contains__("300"):
                        num_vertices = 300
                        instance_info = instance.split(".clq")[0]
                        break
                    elif instance.__contains__("400"):
                        num_vertices = 400
                        instance_info = instance.split(".clq")[0]
                        break
                    elif instance.__contains__("500"):
                        num_vertices = 500
                        instance_info = instance.split(".clq")[0]
                        break

                vertices_w, edges = gen_graph_from_edges(instance_path)

                num_vertices = len(vertices_w)
                num_edges = len(edges)

                start = time()
                set, weight = greedy_mwds(vertices_w, edges)
                end = time()
                elapsed = end - start
                greedy_results = "|Ds| = " + str(len(set)) + "\n" + "weight = " + str(weight) +"\n" + "T(s) = " + f"{elapsed:.3f}"
                print("Greedy ", instance, " done")

                start = time()
                set, weight = lp_model(vertices_w, edges)
                end = time()
                elapsed = end - start
                ilp_results = "|Ds| = " + str(len(set)) + "\n" + "weight = " + str(weight) +"\n" + "T(s) = " + f"{elapsed:.3f}"
                print("PuLP ", instance, " done")


                solutions = []
                times = []
                for i in range(10):
                    ga = GeneticAlgorithm(vertices_w, edges, population_size=100, chromosome_length=num_vertices, mutation_rate=0.05, n_generations=100, max_time=600)
                    start = time()
                    solution = ga.run()
                    end = time()
                    solutions.append(solution)
                    times.append(end - start)

                ds = [s.fitness[0] for s in solutions]
                min_index = ds.index(min(ds))
                min_wds = len([i for i in solutions[min_index].chromosome if i == 1])
                ga_results = "|Ds| = " + str(min_wds) + "\nmean = " + str(mean(ds)) + "\nmax = " + str(max(ds)) + "\nmin =" + str(min(ds)) + "\nsd = " + str(round(std(ds), 3)) +"\nmean(T) = ", str(round(mean(times), 3))

                print("GA ", instance, " done")

                instance_info += str("\n|V| = ") + str(num_vertices) + "\n|E| = " + str(num_edges)
                writer.writerow([instance_info, greedy_results, ga_results, ilp_results])


NameError: name 'dir_instances' is not defined